# Lab02 Part 1: Working LAS Files

By Rob Hendrickson

In [1]:
# Import Libraries

# System

import os # For working with Operating System
import requests # For accessing websites
import zipfile # For extracting from Zipfiles
from io import BytesIO # For reading bytes objects

# Analysis

# import laspy # Working with LAS files
import numpy as np # Numerical Python

# Arcpy Stuff

import arcpy # Arcpy

# Set Working Directory (Arcpy)

arcpy.env.workspace = os.getcwd() + 'Arc1_Lab02.gdb'

## 1

Describe and build an ETL in ArcPro Jupyter Notebooks and arcpy that:

a) Downloads .LAS files from MN DNR [1](https://resources.gisdata.mn.gov/pub/data/elevation/lidar/)
    
b) Converts the .LAS file into both a DEM and a TIN
    
c) Saves the new DEM and TIN to disk
    
d) Exports PDFs of the DEM and TIN with correct visualization (see [here](https://pro.arcgis.com/en/pro-app/latest/arcpy/mapping/introduction-to-arcpy-mp.htm))

In [2]:
# Download Data

downloaded = True

if not(downloaded):

    url = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-14-05.las'

    response = requests.request("GET", url) # Get request

    # Save

    with open("4342-14-05.las", "wb") as file:
        file.write(response.content)

In [3]:
## Make into workable format using laspy
## see: https://medium.com/spatial-data-science/an-easy-way-to-work-and-visualize-lidar-data-in-python-eed0e028996c

# las = laspy.read('4342-14-05.las')

## Make into numpy 3d point cloud tensor

# point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))

## Unique classifications?
# classifications = np.unique(np.array(las.classification))

In [ ]:
# Load into arcpy... As multipoint

filename = "4342-14-05.las"

arcpy.ddd.LASToMultipoint(filename, 'las_example_multipoint')

# Create Terrain

arcpy.ddd.CreateTerrain('las_example_multipoint', 'las_example_terrain')



## 2

Do side-by-side exploratory data analysis with a 2D map of the .las file on one
pane and a 3D Scene of the .las file on another pane. This will be very
computationally intensive, so use a small .las file. In your writeup, describe the
features provided by ArcGIS for working with 2D and 3D visualization of .las files.

## 3

Describe and build an ETL in ArcPro Jupyter Notebooks that:

a. Downloads the annual 30-Year Normals .bil files for precipitation from
PRISM [2](https://prism.oregonstate.edu/normals/)

- See https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf for details
- File we want: PRISM_ppt_30yr_normal_4kmM3_annual_bil.zip
- Careful here! You cannot download the data more than once per day!!

b. Converts the data into a spacetime cube and exports it to disk (see here
for example of final conversion step; to get to this point, you will need to
go through other transformation steps likely) [3](https://www.esri.com/arcgis-blog/products/arcgis-pro/analytics/explore-your-raster-data-with-space-time-pattern-mining/)

c. Export an animation of the timeseries

In [4]:
# Download Data

downloaded = True # Downloaded?

# Make folder for .bil files

folder_name = 'PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month'
path = os.path.join(os.getcwd(), folder_name)
if folder_name not in os.listdir():
    os.mkdir(path)
    
# Download
    
if not(downloaded):

    for month in range(1,13): # Iterate through months

        url = 'http://services.nacse.org/prism/data/public/normals/4km/ppt/' + str(month) # Url for month

        response = requests.request("GET", url) # Get request
        
        # Unload zip into the folder
        
        if len(response.text) == 201: # Too many requests
            print('Failed to extract...')
            print(response.text)
        else: 
            zip_folder = zipfile.ZipFile(BytesIO(response.content)) # Read Response
            zip_folder.extractall(path=path) # Extract files
            zip_folder.close() # Close zip object

In [13]:
# Add all as Arcpy rasters into a list

bils_files = os.listdir(folder_name) # Some of these are metadata, etc.

rasts = [] # Initialize raster storage
feature_names = [] # Initialize feature_name storage
m = 0 # Month Counter

for file in bils_files:
    if file[-4:] == '.bil': # Only want .bil files
        path = os.path.join(folder_name, file)
        
        rasts += [arcpy.Raster(path)] # Store Arcpy Raster in list
        # Arcpy Raster objects: https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/raster-object.htm
                
        feature_names += [file.split('.')[0][:-4]] # Store feature_name
        
        m += 1 # Next month...



In [11]:
rasts

[PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_01_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_02_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_03_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_04_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_05_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_06_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_07_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_08_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_09_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_10_bil.bil, PRISM_ppt_30yr_normal_4kmM3_annual_bils_by_month\PRISM_ppt_30yr_normal_4kmM3_11_bil.bil, PRISM_ppt_30yr_norma

In [14]:
feature_names

['PRISM_ppt_30yr_normal_4kmM3_01', 'PRISM_ppt_30yr_normal_4kmM3_02', 'PRISM_ppt_30yr_normal_4kmM3_03', 'PRISM_ppt_30yr_normal_4kmM3_04', 'PRISM_ppt_30yr_normal_4kmM3_05', 'PRISM_ppt_30yr_normal_4kmM3_06', 'PRISM_ppt_30yr_normal_4kmM3_07', 'PRISM_ppt_30yr_normal_4kmM3_08', 'PRISM_ppt_30yr_normal_4kmM3_09', 'PRISM_ppt_30yr_normal_4kmM3_10', 'PRISM_ppt_30yr_normal_4kmM3_11', 'PRISM_ppt_30yr_normal_4kmM3_12']

In [19]:
# Create point clouds...

cloud_filenames = [] # Initialize filename storage

for m, rast in enumerate(rasts):

    # Pt Cloud Filename
    
    new_name = os.path.join('Pt_Clouds', feature_names[m] + '_pts.shp')
    cloud_filenames += [new_name]
    
    # Create Point Cloud

    arcpy.conversion.RasterToPoint(rast, new_name) # Raster to Points

    arcpy.management.AddField(new_name, 'Date', 'DATE') # Create month field in pointcloud
    
    m_string = feature_names[m][-2:]
    
    date = '2020-' + m_string + '-01'
    
    expression = "date()"
    code_block = f"""
def date():
    return '{date}'
"""
    
    arcpy.management.CalculateField(new_name, 'Date', expression, 'PYTHON3', code_block) # Add month value

In [20]:
# Merge all point clouds (Think about clearing scratch gdb)

# initialize feature class

all_clouds_filename = feature_names[0][:-10] + '_allmonths_pts.shp' # Name

# merge

arcpy.management.Merge(cloud_filenames, 
                       all_clouds_filename, "", "ADD_SOURCE_INFO")

<Result 'C:\\Users\\hendr862\\Desktop\\Arc1_Lab02_part1\\PRISM_ppt_30yr_normaallmonths_pts.shp'>

In [25]:
all_clouds_filename

'PRISM_ppt_30yr_normaallmonths_pts.shp'

In [30]:
# Convert to SpacetimeCube

spacetime_cube_name = all_clouds_filename[0:-7] + 'spacetimecube'


arcpy.geoanalytics.CreateSpaceTimeCube(all_clouds_filename,
                               spacetime_cube_name,
                                '1 Kilometers',
                                '1 Months'
                              )


Traceback (most recent call last):
  File "c:\program files\arcgis\pro\Resources\ArcToolbox\scripts\ga_server_createspacetimecube.py", line 49, in <module>
    output = ga.run_portal_tool(params)
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcToolbox\Scripts\geoanalyticssoap.py", line 93, in run_portal_tool
    self.add_toolbox()
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcToolbox\Scripts\geoanalyticssoap.py", line 77, in add_toolbox
    arcpy.ImportToolbox(self.toolbox)
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\__init__.py", line 171, in ImportToolbox
    return import_toolbox(input_file, module_name)
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\toolbox_code.py", line 486, in import_toolbox
    toolbox = gp.createObject("Toolbox", tbxfile)
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\geoprocessing\_base.py", line 389, in createObject
    self._gp.CreateObject(*gp_fixargs(args, True)))
OSError: The toolbox file https://geoanalytics.arcgi

ExecuteError: ERROR 000582: Error occurred during execution.
